1. 数组中第K大的元素

In [ ]:
import heapq

nums = [1,2,3,4,5,6,7,8,9]
k = 3

min_heap = []
for i, num in enumerate(nums):
    if i < k:
        heapq.heappush(min_heap, num)
    else:
        if num > min_heap[0]:
            heapq.heappop(min_heap)
            heapq.heappush(min_heap, num)
print(min_heap[0])

7


2. 最长不重复子串

In [3]:
str = "abcabcbb"
left = 0 
last_occur = {}
max_len = 0
for right, char in enumerate(str):
    if char in last_occur and last_occur[char] >= left:
        left = last_occur[char] + 1
    last_occur[char] = right
    str_len = right - left + 1
    if str_len > max_len:
        max_len = str_len
print(max_len)

3


3. 实现Multi-head Attention

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads

        assert (
            self.head_dim * num_heads == embed_size
        ), "Embedding size needs to be divisible by num_heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(num_heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into `num_heads` different pieces
        values = values.reshape(N, value_len, self.num_heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.num_heads, self.head_dim)
        queries = query.reshape(N, query_len, self.num_heads, self.head_dim)

        values = self.values(values)  # (N, value_len, num_heads, head_dim)
        keys = self.keys(keys)  # (N, key_len, num_heads, head_dim)
        queries = self.queries(queries)  # (N, query_len, num_heads, head_dim)

        # Einsum does batched matrix multiplication for query*keys for each training example
        # with shape (N, num_heads, query_len, key_len)
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.num_heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

4. 多面骰子n次和大于等于k的概率

In [22]:
def calculate_probobility(m, target):
    dp = [[0] * (m * 6 + 1) for i in range(m)]
    for i in range(1, 7):
        dp[0][i] = 1/6
    for i in range(1, m):
        for j in range(1, i * 6 + 1):
            for k in range(1, 7):
                dp[i][j + k] += dp[i-1][j] / 6
    return sum(dp[m-1][target:])

print(calculate_probobility(2, 15))

0


5. 字母金字塔

In [4]:
def left_padding(s, n):
    return ' ' * (n - len(s)) + s

def reverse(s):
    return s[::-1]

def alphabeta_pyramid(s):
    n = len(s)
    for i in range(n):
        s_ = left_padding(s[:i+1], n)
        s_ = s_ + reverse(s_[:-1])
        print(s_)

alphabeta_pyramid("ABCDE")

    A    
   ABA   
  ABCBA  
 ABCDCBA 
ABCDEDCBA
